In [0]:
import pandas as pd
import pandas_profiling
import numpy as np
import nltk 
import re

from tqdm import tqdm
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelEncoder, StandardScaler
from textblob import TextBlob
from scipy.sparse import hstack, csr_matrix
from sklearn.model_selection import train_test_split                                          
from sklearn.svm import LinearSVR
from sklearn.metrics import mean_absolute_error


nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

Reading the data.

In [0]:
train = pd.read_csv('/content/drive/My Drive/zs_news_data/dataset/train_file.csv')
test = pd.read_csv('/content/drive/My Drive/zs_news_data/dataset/test_file.csv')

Let's get some insights into the data.
Exploratory Data Analysis(EDA)

In [0]:
train.head()

,IDLink,Title,Headline,Source,Topic,PublishDate,Facebook,GooglePlus,LinkedIn,SentimentTitle,SentimentHeadline
0,Tr3CMgRv1N,Obama Lays Wreath at Arlington National Cemetery,Obama Lays Wreath at Arlington National Cemete...,USA TODAY,obama,2002-04-02 00:00:00,-1,-1,-1,0.000000,-0.053300
1,Wc81vGp8qZ,A Look at the Health of the Chinese Economy,"Tim Haywood, investment director business-unit...",Bloomberg,economy,2008-09-20 00:00:00,-1,-1,-1,0.208333,-0.156386
2,zNGH03CrZH,Nouriel Roubini: Global Economy Not Back to 2008,"Nouriel Roubini, NYU professor and chairman at...",Bloomberg,economy,2012-01-28 00:00:00,-1,-1,-1,-0.425210,0.139754
3,3sM1H0W8ts,Finland GDP Expands In Q4,Finland's economy expanded marginally in the t...,RTT News,economy,2015-03-01 00:06:00,-1,-1,-1,0.000000,0.026064
4,wUbnxgvqaZ,"Tourism, govt spending buoys Thai economy in J...",Tourism and public spending continued to boost...,The Nation - Thailand&#39;s English news,economy,2015-03-01 00:11:00,-1,-1,-1,0.000000,0.141084


In [0]:
test.head()

,IDLink,Title,Headline,Source,Topic,PublishDate,Facebook,GooglePlus,LinkedIn
0,tFrqIR6Chj,Sliding Economy: FG fights back with N3trn TSA...,With the 2016 budget now passed by the Nationa...,BusinessDay,economy,2016-03-29 01:41:12,0,0,1
1,DVAaGErjlF,Microsoft shows how HoloLens can bring distant...,A recent Microsoft Research video shows how th...,Daily Mail,microsoft,2016-03-29 01:41:27,121,2,13
2,OT9UIZm5M2,"Microsoft’s Twitter Robot Praises Hitler, Trum...",* Microsoft teamed with Bing to create TayTwee...,EURweb,microsoft,2016-03-29 01:47:00,12,1,0
3,lflGp3q2Fj,Flood of Central Bank Moves Can't Get World Ec...,Central bankers have managed to steer the worl...,Bloomberg via Yahoo! Finance,economy,2016-03-29 02:00:00,0,0,3
4,zDYG0SoovZ,USD/JPY: bears lining up on mixed U.S. economy...,"However, this streak of seven-day gains might ...",FXStreet,economy,2016-03-29 02:01:07,3,0,0


In [0]:
train.describe()

,Facebook,GooglePlus,LinkedIn,SentimentTitle,SentimentHeadline
count,55932.000000,55932.000000,55932.000000,55932.000000,55932.000000
mean,132.050329,4.551616,14.300132,-0.006318,-0.029577
std,722.931314,21.137177,76.651420,0.137569,0.143038
min,-1.000000,-1.000000,-1.000000,-0.838525,-0.755355
25%,0.000000,0.000000,0.000000,-0.079057,-0.116927
50%,6.000000,0.000000,0.000000,0.000000,-0.027277
75%,37.000000,2.000000,4.000000,0.063969,0.057354
max,49211.000000,1267.000000,3716.000000,0.962354,0.964646


In [0]:
pandas_profiling.ProfileReport(train)

/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,11
Number of observations,55932
Total Missing (%),0.0%
Total size in memory,4.7 MiB
Average record size in memory,88.0 B
Numeric,5
Categorical,5
Boolean,0
Date,0
Text (Unique),1
Rejected,0


From the above profiling we see that there only one column that has missing values i.e. Source  

In [0]:
train.isnull().sum()  #just crosss checking 

IDLink                 0
Title                  0
Headline               0
Source               175
Topic                  0
PublishDate            0
Facebook               0
GooglePlus             0
LinkedIn               0
SentimentTitle         0
SentimentHeadline      0
dtype: int64

In [0]:
test.isnull().sum() #test column also has missing values in the source column only

IDLink           0
Title            0
Headline         0
Source         101
Topic            0
PublishDate      0
Facebook         0
GooglePlus       0
LinkedIn         0
dtype: int64

In [0]:
train['Source'].value_counts()[:5]    #Top 5 values  

Bloomberg         992
Reuters           763
ABC News          645
New York Times    573
The Guardian      551
Name: Source, dtype: int64

In [0]:
train['Source'] = train['Source'].fillna('Bloomberg')
test['Source'] = test['Source'].fillna('Bloomberg')

Let's check for duplicates as well

In [0]:
duplicateRowstrain = train[train.duplicated()] 
duplicateRowstrain

,IDLink,Title,Headline,Source,Topic,PublishDate,Facebook,GooglePlus,LinkedIn,SentimentTitle,SentimentHeadline


Now we will proceed to Text Preprocessing and Cleaning.

In [0]:
stop_words = set(stopwords.words('english'))

def cleaning_text(text):
  text_token = word_tokenize(text)
  filtered_text = ' '.join([w.lower() for w in text_token if w.lower() not in stop_words and len(w) > 2])
  filtered_text = filtered_text.replace(r"[^a-zA-Z]+", '')
  filtered_text = re.sub(r'\b\d+\b', '', filtered_text)
  clean_text = filtered_text.replace(',', '').replace('.', '').replace(':', '')
  return clean_text

In [0]:
train['Title_Source_Topic'] = train['Title']+ ' ' + train['Source'] + ' ' + train['Topic']
test['Title_Source_Topic'] = test['Title']+ ' ' + test['Source'] + ' ' + test['Topic']

train['Headline_Source_Topic'] = train['Headline'] + ' ' + train['Source'] + ' ' + train['Topic']
test['Headline_Source_Topic'] = test['Headline'] + ' ' + test['Source'] + ' ' + test['Topic']

In [0]:
train['Title_Source_Topic']

0        Obama Lays Wreath at Arlington National Cemete...
1        A Look at the Health of the Chinese Economy Bl...
2        Nouriel Roubini: Global Economy Not Back to 20...
3               Finland GDP Expands In Q4 RTT News economy
4        Tourism, govt spending buoys Thai economy in J...
                               ...                        
55927    Fidel Castro Lashes Out at Obama After Cuba Vi...
55928    JOHN CRISP | Obama's strategic reaction to Bru...
55929    Think Trump's 45 Percent Tariffs Are Bad? Try ...
55930    Microsoft finally releases giant Surface TechE...
55931    Rays of sunshine in the US economy Washington ...
Name: Title_Source_Topic, Length: 55932, dtype: object

In [0]:
train['Title_Source_Topic'][4]

'Tourism, govt spending buoys Thai economy in January The Nation - Thailand&#39;s English news economy'

In [0]:
train.tail()

,IDLink,Title,Headline,Source,Topic,PublishDate,Facebook,GooglePlus,LinkedIn,SentimentTitle,SentimentHeadline,Title_Source_Topic,Headline_Source_Topic
55927,jQ3CeLRCb9,Fidel Castro Lashes Out at Obama After Cuba Visit,Retired Cuban leader Fidel Castro slammed Pres...,Wall Street Journal,obama,2016-03-29 01:35:06,794,10,5,-0.135417,-0.055902,Fidel Castro Lashes Out at Obama After Cuba Vi...,Retired Cuban leader Fidel Castro slammed Pres...
55928,akNYeJ8opY,JOHN CRISP | Obama's strategic reaction to Bru...,President Obama caught some predictable flak f...,Kitsap Sun,obama,2016-03-29 01:35:08,0,0,0,0.236228,0.056110,JOHN CRISP | Obama's strategic reaction to Bru...,President Obama caught some predictable flak f...
55929,n2DGs0c8IG,Think Trump's 45 Percent Tariffs Are Bad? Try ...,While Trump wants to put large tariffs on impo...,Huffington Post,obama,2016-03-29 01:35:09,102,4,0,0.025747,0.114820,Think Trump's 45 Percent Tariffs Are Bad? Try ...,While Trump wants to put large tariffs on impo...
55930,P0EBiaSEjq,Microsoft finally releases giant Surface,Microsoft’s business customers are finally beg...,TechEye,microsoft,2016-03-29 01:38:00,0,0,0,0.000000,-0.028296,Microsoft finally releases giant Surface TechE...,Microsoft’s business customers are finally beg...
55931,99MLvyAQTJ,Rays of sunshine in the US economy,AS WE all know from listening to the campaign ...,Washington Post,economy,2016-03-29 01:41:08,75,7,19,0.000000,0.184444,Rays of sunshine in the US economy Washington ...,AS WE all know from listening to the campaign ...


In [0]:
for x in train['Title_Source_Topic']:
  print (x)

Streaming output truncated to the last 5000 lines.
Economic Growth through Crisis EconomyWatch.com economy
Microsoft adds Touch ID support to Outlook for iOS ITProPortal microsoft
Exit polls: Economy, feeling of betrayal CNN.com economy
Microsoft highlights February's updates to Office 365 WinBeta microsoft
Investment Opportunities Around China’s Weakening Economy Investopedia economy
Microsoft announces quarterly dividend PR Newswire via Yahoo! Finance microsoft
What Is Sony's Incentive To Play Nice With Microsoft For PS4-Xbox ... Forbes microsoft
Halperin: Trump More Talented At Running For President Than ... Daily Caller obama
Microsoft announces developer preview of new Bing Search APIs MSPoweruser.com microsoft
RBA content with Aussie economy - UOB FXStreet economy
Lin-Manuel Miranda, 'Hamilton' Creator, Freestyles for Obama New York Times obama
Microsoft announces quarterly dividend PR Newswire microsoft
President Obama Calls 'Vicious' 2016 Campaign a 'Race to the ... ABC News ob

In [0]:
train['Title_Source_Topic'] = [cleaning_text(x) for x in tqdm(train['Title_Source_Topic'])]
test['Title_Source_Topic'] = [cleaning_text(x) for x in tqdm(test['Title_Source_Topic'])]

train['Headline_Source_Topic'] = [cleaning_text(x) for x in tqdm(train['Headline_Source_Topic'])]
test['Headline_Source_Topic'] = [cleaning_text(x) for x in tqdm(test['Headline_Source_Topic'])]

100%|██████████| 37288/37288 [00:11<00:00, 3376.57it/s]


In [0]:
vect = TfidfVectorizer(use_idf= True)

In [0]:
train_vect_tts = vect.fit_transform(train['Title_Source_Topic'])
test_vect_tts = vect.transform(test['Title_Source_Topic'])

In [0]:
vect_2 = TfidfVectorizer()

In [0]:
train_vect_2_hst = vect_2.fit_transform(train['Headline_Source_Topic'])
test_vect_2_hst = vect_2.transform(test['Headline_Source_Topic'])

In [0]:
train['polarity_title'] = train['Title'].apply(lambda x: TextBlob(x).sentiment.polarity)
test['polarity_title'] = test['Title'].apply(lambda x: TextBlob(x).sentiment.polarity)

train['subjectivity_title'] = train['Title'].apply(lambda x: TextBlob(x).sentiment.subjectivity)
test['subjectivity_title'] = test['Title'].apply(lambda x: TextBlob(x).sentiment.subjectivity)

In [0]:
train['polarity_headline'] = train['Headline'].apply(lambda x: TextBlob(x).sentiment.polarity)
test['polarity_headline'] = test['Headline'].apply(lambda x: TextBlob(x).sentiment.polarity)

train['subjectivity_headline'] = train['Headline'].apply(lambda x: TextBlob(x).sentiment.subjectivity)
test['subjectivity_headline'] = test['Headline'].apply(lambda x: TextBlob(x).sentiment.subjectivity)

In [0]:
# Encoding the categorical variables
encoder = LabelEncoder()

train['Topic'] = encoder.fit_transform(train['Topic'])
test['Topic'] = encoder.transform(test['Topic'])

total = train['Source'].to_list() + test['Source'].to_list()
total = encoder.fit_transform(total)
train['Source'] = encoder.transform(train['Source'])
test['Source'] = encoder.transform(test['Source'])

In [0]:
#Feature scaling

scaler = StandardScaler()

cols = ['Source', 'Topic', 'Facebook', 'GooglePlus', 'LinkedIn']

for col in cols:
  train[col] = scaler.fit_transform(train[col].values.reshape(-1, 1))
  test[col] = scaler.transform(test[col].values.reshape(-1, 1))

In [0]:
train.loc[[1185]]

,IDLink,Title,Headline,Source,Topic,PublishDate,Facebook,GooglePlus,LinkedIn,SentimentTitle,SentimentHeadline,Title_Source_Topic,Headline_Source_Topic,polarity_title,subjectivity_title,polarity_headline,subjectivity_headline
1185,N6BjAKDaYw,UPDATE 2-Saudi Arabia says will diversify oil ...,Last of Group of 20 major economies to submit ...,0.508623,-1.108773,2015-11-10 15:00:09,-0.171595,-0.168028,-0.186562,-0.128468,0.128108,update -saudi arabia says diversify oil econom...,last group major economies submit plan adds ba...,-0.3,0.4,0.03125,0.283333


In [0]:
cols_title = ['Source', 'Topic', 'Facebook', 'GooglePlus', 'LinkedIn','polarity_title', 'subjectivity_title']
train_title = train[cols_title]
test_title = test[cols_title]

cols_headline = ['Source', 'Topic', 'Facebook', 'GooglePlus', 'LinkedIn','polarity_headline', 'subjectivity_headline']
train_headline = train[cols_headline]
test_headline = test[cols_headline]

In [0]:
train_title.head()

,Source,Topic,Facebook,GooglePlus,LinkedIn,polarity_title,subjectivity_title
0,1.257700,0.841443,-0.184044,-0.262649,-0.199608,0.0,0.0
1,-1.314091,-1.108773,-0.184044,-0.262649,-0.199608,0.0,0.0
2,-1.314091,-1.108773,-0.184044,-0.262649,-0.199608,0.0,0.0
3,0.468412,-1.108773,-0.184044,-0.262649,-0.199608,0.0,0.0
4,1.029071,-1.108773,-0.184044,-0.262649,-0.199608,0.0,0.0


In [0]:
test_title.head()

,Source,Topic,Facebook,GooglePlus,LinkedIn,polarity_title,subjectivity_title
0,-1.233094,-1.108773,-0.182661,-0.215339,-0.173516,0.0,0.00
1,-0.953339,-0.133665,-0.015286,-0.120718,-0.016962,-0.1,0.35
2,-0.836153,-0.133665,-0.166062,-0.168028,-0.186562,0.0,0.00
3,-1.310644,-1.108773,-0.182661,-0.215339,-0.147424,0.0,0.25
4,-0.717817,-1.108773,-0.178511,-0.215339,-0.186562,0.0,0.25


In [0]:
train_headline.head()

,Source,Topic,Facebook,GooglePlus,LinkedIn,polarity_headline,subjectivity_headline
0,1.257700,0.841443,-0.184044,-0.262649,-0.199608,0.000000,0.000000
1,-1.314091,-1.108773,-0.184044,-0.262649,-0.199608,0.100000,0.200000
2,-1.314091,-1.108773,-0.184044,-0.262649,-0.199608,0.000000,0.041667
3,0.468412,-1.108773,-0.184044,-0.262649,-0.199608,-0.166667,0.166667
4,1.029071,-1.108773,-0.184044,-0.262649,-0.199608,0.133333,0.380556


In [0]:
test_headline.head()

,Source,Topic,Facebook,GooglePlus,LinkedIn,polarity_headline,subjectivity_headline
0,-1.233094,-1.108773,-0.182661,-0.215339,-0.173516,0.000000,0.000000
1,-0.953339,-0.133665,-0.015286,-0.120718,-0.016962,0.000000,0.250000
2,-0.836153,-0.133665,-0.166062,-0.168028,-0.186562,-0.157407,0.251852
3,-1.310644,-1.108773,-0.182661,-0.215339,-0.147424,0.087500,0.272917
4,-0.717817,-1.108773,-0.178511,-0.215339,-0.186562,0.000000,0.000000


In [0]:
print(np.shape(train_title))
print(np.shape(test_title))

(55932, 7)
(37288, 7)


In [0]:
print(np.shape(train_headline))
print(np.shape(test_headline))

(55932, 7)
(37288, 7)


In [0]:
print(np.shape(train_vect_tts))
print(np.shape(test_vect_tts))

(55932, 25377)
(37288, 25377)


In [0]:
print(np.shape(train_vect_2_hst))
print(np.shape(test_vect_2_hst))

(55932, 38787)
(37288, 38787)


In [0]:
train_X_Title = hstack([train_vect_tts, csr_matrix(train_title.values)])
test_X_Title = hstack([test_vect_tts, csr_matrix(test_title.values)])
y1 = train['SentimentTitle']

In [0]:
train_X_Headline = hstack([train_vect_2_hst, csr_matrix(train_headline.values)])
test_X_Headline = hstack([test_vect_2_hst, csr_matrix(test_headline.values)])
y2 = train['SentimentHeadline']

In [0]:
np.shape(train_X_Title)

(55932, 25384)

In [0]:
#model for sentiment title
X_train, X_test, y_train, y_test = train_test_split(train_X_Title, y1, test_size=0.20, random_state=42)

LSVR1 = LinearSVR(C=0.2)
LSVR1.fit(X_train, y_train)

y_pred1 = LSVR1.predict(X_test)
mae1 = mean_absolute_error(y_pred1, y_test)
print('MAE:', 1 - mae1)

MAE: 0.9433410007252648


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [0]:
X_train, X_test, y_train, y_test = train_test_split(train_X_Headline, y2, test_size=0.20, random_state=42)

LSVR2 = LinearSVR(C=0.1)
LSVR2.fit(X_train, y_train)

y_pred2 = LSVR2.predict(X_test)
mae2 = mean_absolute_error(y_pred2, y_test)
print('MAE:', 1 - mae2)

MAE: 0.9291880665306038


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [0]:
print('MAE:', 1 - ((0.4 * mae1) + (0.6 * mae2)))

MAE: 0.9348492402084683


In [0]:
pred_title = LSVR1.predict(test_X_Title)
pred_headline = LSVR2.predict(test_X_Headline)

In [0]:
test_id = test['IDLink']

In [0]:
df = pd.DataFrame()
df['IDLink'] = test_id
df['SentimentTitle'] = pred_title
df['SentimentHeadline'] = pred_headline
df.to_csv('/content/drive/My Drive/zs_news_data/submission_LSVR.csv', index=False)